# Topic Modeling with LDA

In this notebook, I'll take my processed coca data (punctuation removed; stopwords removed; no links, hashtags, or emojis; and lemmatized) and use pyLDAvis to start exploring some of the underlying topics in this corpus. 

pyLDAvis is a great tool to both get top keywords for each topic as well as *visualize* these topics in relation to one another. 

## pyLDAvis Topic Modeling

In [26]:
# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#sklearn
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import warnings
warnings.filterwarnings('ignore')

import pyLDAvis
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()

import getout_of_text_3 as got3

from tqdm.notebook import tqdm
from tqdm import tqdm

## Read the COCA text using `got3`

In [2]:
coca_corpus = got3.read_corpus('../coca-text/')


Genres:   0%|          | 0/8 [00:00<?, ?genre/s]

Processing genre: mag


Genres:  12%|█▎        | 1/8 [00:00<00:06,  1.02genre/s]

Finished genre: mag (total files: 30)
Processing genre: web


Genres:  25%|██▌       | 2/8 [00:01<00:05,  1.02genre/s]

...

Finished genre: blog (total files: 34)
Processing genre: fic


Genres:  88%|████████▊ | 7/8 [00:06<00:00,  1.08genre/s]

Finished genre: fic (total files: 30)
Processing genre: tvm


Genres: 100%|██████████| 8/8 [00:07<00:00,  1.05genre/s]

Finished genre: tvm (total files: 30)


### Search for a keyword to find topic distributions

The classic example of `bank` whether it's a financial bank or a river bank. We get kwic results from COCA for `bank` and then use LDA

In [9]:
before = pd.Timestamp.now()
bovine_kwic = got3.search_keyword_corpus('bank', coca_corpus, 
                                            case_sensitive=False,
                                            show_context=False, 
                                            context_words=15,
                                            output='print',
                                            parallel=True)
after = pd.Timestamp.now()
print('time elapsed:', after - before)

🔍 COCA Corpus Search: 'bank'
🚀 Using parallel processing with 9 processes...

📚 MAG_1993 :
------------------------------
  ✅ Found 423 occurrence(s) in mag_1993

📚 MAG_1992 :
------------------------------
  ✅ Found 764 occurrence(s) in mag_1992

📚 MAG_1990 :
------------------------------
  ✅ Found 1075 occurrence(s) in mag_1990

📚 MAG_1991 :
------------------------------
  ✅ Found 1062 occurrence(s) in mag_1991

📚 MAG_1995 :
------------------------------
  ✅ Found 489 occurrence(s) in mag_1995

📚 MAG_1994 :
------------------------------
  ✅ Found 468 occurrence(s) in mag_1994

📚 MAG_1996 :
------------------------------
  ✅ Found 460 occurrence(s) in mag_1996

📚 MAG_1997 :
------------------------------
  ✅ Found 456 occurrence(s) in mag_1997

📚 MAG_2008 :
------------------------------
  ✅ Found 434 occurrence(s) in mag_2008

📚 MAG_2009 :
------------------------------
  ✅ Found 446 occurrence(s) in mag_2009

📚 MAG_2019 :
------------------------------
  ✅ Found 456 occurrence(s

### Lemmatize and clean the kwic results

In [4]:
import re
import string
from typing import Iterable, List, Dict, Any, Optional

# If you already have a stopword set defined elsewhere you can pass it in; otherwise we build one.
def build_stopwords(extra: Optional[Iterable[str]] = None):
    try:
        from nltk.corpus import stopwords
        try:
            _ = stopwords.words('english')
        except LookupError:
            import nltk
            nltk.download('stopwords')
        base = set(stopwords.words('english'))
    except Exception:
        # Fallback minimal list if nltk not available
        base = {
            'the','a','an','and','or','to','of','in','on','for','at','by','with','from','that',
            'this','is','was','are','were','be','been','it','as','but','if','than','then','so',
            'such','into','its','their','there','here','over','under','out','up','down','not'
        }
    if extra:
        base.update(w.lower() for w in extra)
    return base

def load_spacy(model_name: str = "en_core_web_sm"):
    """Load spaCy model (download if missing)."""
    import importlib
    try:
        import spacy
        return spacy.load(model_name)
    except OSError:
        try:
            from spacy.cli import download
            download(model_name)
            import spacy
            from tqdm import tqdm
            return spacy.load(model_name)
        except Exception as e:
            raise RuntimeError(f"Could not load or download spaCy model '{model_name}': {e}")

# Precompile regexes
URL_RE = re.compile(r'https?://\\S+|www\\.\\S+', re.IGNORECASE)
HASHTAG_RE = re.compile(r'#[A-Za-z0-9_]+')
HANDLE_RE = re.compile(r'@[A-Za-z0-9_]+')
# Basic emoji / symbol range (broad)
EMOJI_RE = re.compile(r'[\\U00010000-\\U0010ffff]', flags=re.UNICODE)
# Markdown bold markers **word**
BOLD_MARK_RE = re.compile(r'\\*\\*')

# Punctuation translator (remove all punctuation chars)
PUNCT_TABLE = str.maketrans('', '', string.punctuation)

def clean_context(text: str,
                  nlp=None,
                  stopwords_set: Optional[set] = None,
                  keep_keyword: Optional[str] = None,
                  min_len: int = 2) -> str:
    """
    Clean a single KWIC context string.
    
    Steps:
      - Lowercase
      - Remove markdown bold markers (**)
      - Remove URLs, hashtags, handles
      - Remove emojis (basic range)
      - Strip any remaining asterisks
      - Remove punctuation
      - Tokenize (simple split after punctuation removal)
      - Drop stopwords & short tokens
      - Lemmatize (spaCy) if nlp supplied
      - Optionally keep the keyword even if filtered
    
    Returns a space-separated string of lemmas/tokens.
    """
    if not isinstance(text, str):
        text = str(text)
    if stopwords_set is None:
        stopwords_set = build_stopwords()
    lowered = text.lower()
    lowered = BOLD_MARK_RE.sub('', lowered)  # remove ** markers
    lowered = URL_RE.sub(' ', lowered)
    lowered = HASHTAG_RE.sub(' ', lowered)
    lowered = HANDLE_RE.sub(' ', lowered)
    lowered = EMOJI_RE.sub(' ', lowered)
    # Remove any leftover asterisks (in case of unmatched markdown)
    lowered = lowered.replace('*', ' ')
    # Replace newlines with space
    lowered = lowered.replace('\\n', ' ')
    # Remove punctuation
    no_punct = lowered.translate(PUNCT_TABLE)
    # Collapse multiple spaces
    no_punct = re.sub(r'\\s+', ' ', no_punct).strip()
    if not no_punct:
        return ''
    tokens = no_punct.split()
    if nlp is not None:
        # Lemmatize
        doc = nlp(' '.join(tokens))
        lemmas = []
        for token in doc:
            lemma = token.lemma_.lower()
            # Remove pronoun artifacts
            if lemma == '-pron-':
                continue
            lemmas.append(lemma)
        tokens = lemmas
    cleaned = []
    keep_kw = keep_keyword.lower() if keep_keyword else None
    for tok in tokens:
        if keep_kw and tok == keep_kw:
            cleaned.append(tok)
            continue
        if tok in stopwords_set:
            continue
        if len(tok) < min_len:
            continue
        if tok.isnumeric():
            continue
        cleaned.append(tok)
    return ' '.join(cleaned)

def kwic_dict_to_docs_seq(kwic_dict: Dict[str, Any],
                      nlp=None,
                      stopwords_set: Optional[set] = None,
                      keep_keyword: Optional[str] = None,
                      min_len: int = 2,
                      show_progress: bool = False) -> List[str]:
    """
    Flatten the kwic result (genre_year -> list[dict{'context': ...}]) into a list of cleaned strings.
    Skips empty cleaned results.
    If show_progress is True, uses tqdm to show a progress bar.
    """
    docs = []
    items = list(kwic_dict.items())
    iterator = items
    if show_progress:
        iterator = tqdm(items, desc="Cleaning KWIC contexts")
    for genre_year, entries in tqdm(iterator, desc="Cleaning KWIC contexts", total=len(items)):
        if isinstance(entries, list):
            for item in entries:
                ctx = item.get('full_text') if isinstance(item, dict) else str(item)
                cleaned = clean_context(ctx, nlp=nlp, stopwords_set=stopwords_set,
                                        keep_keyword=keep_keyword, min_len=min_len)
                if cleaned:
                    docs.append(cleaned)
        else:
            # Unexpected structure fallback
            ctx = str(entries)
            cleaned = clean_context(ctx, nlp=nlp, stopwords_set=stopwords_set,
                                    keep_keyword=keep_keyword, min_len=min_len)
            if cleaned:
                docs.append(cleaned)
    return docs


In [8]:
import multiprocessing
from functools import partial
from concurrent.futures import ProcessPoolExecutor
import os

from typing import Dict, Any, Optional, List

def kwic_dict_to_docs(kwic_dict: Dict[str, Any],
                      nlp=None,
                      stopwords_set: Optional[set] = None,
                      keep_keyword: Optional[str] = None,
                      min_len: int = 2,
                      show_progress: bool = False,
                      parallel: bool = False,
                      n_jobs: Optional[int] = None,
                      chunk_size: int = 100) -> List[str]:
    """
    Flatten the kwic result (genre_year -> list[dict{'context': ...}]) into a list of cleaned strings.
    Skips empty cleaned results.
    
    Args:
        kwic_dict: The KWIC results dictionary
        nlp: spaCy model for lemmatization
        stopwords_set: Set of stopwords to filter
        keep_keyword: Keyword to always keep even if filtered
        min_len: Minimum token length
        show_progress: Show progress bar
        parallel: Use multiprocessing for faster processing
        n_jobs: Number of worker processes (None = auto-detect CPU count)
        chunk_size: Size of chunks for multiprocessing
    
    Returns:
        List of cleaned document strings
    """
    # Collect all entries first
    all_entries = []
    for genre_year, entries in kwic_dict.items():
        if isinstance(entries, list):
            all_entries.extend(entries)
        else:
            # Unexpected structure fallback
            all_entries.append(entries)
    
    if not all_entries:
        return []
    
    # Define worker function locally to avoid pickling issues
    def process_entry(entry):
        if isinstance(entry, dict):
            ctx = entry.get('context', entry.get('full_text', ''))
        else:
            ctx = str(entry)
        return clean_context(ctx, nlp=nlp, stopwords_set=stopwords_set,
                            keep_keyword=keep_keyword, min_len=min_len)
    
    if parallel and len(all_entries) > chunk_size:
        try:
            # Use ThreadPoolExecutor instead of ProcessPoolExecutor for Jupyter compatibility
            from concurrent.futures import ThreadPoolExecutor
            
            if n_jobs is None:
                n_jobs = min(4, os.cpu_count() or 1)  # Limit threads for I/O bound tasks
            
            # Process in parallel using threads
            with ThreadPoolExecutor(max_workers=n_jobs) as executor:
                if show_progress:
                    # Submit all tasks and track progress
                    futures = [executor.submit(process_entry, entry) for entry in all_entries]
                    results = []
                    for future in tqdm(futures, desc="Cleaning KWIC contexts (parallel)"):
                        result = future.result()
                        if result:
                            results.append(result)
                else:
                    # Process without progress tracking
                    futures = [executor.submit(process_entry, entry) for entry in all_entries]
                    results = [future.result() for future in futures if future.result()]
            
            return results
            
        except Exception as e:
            print(f"Parallel processing failed ({e}), falling back to sequential processing...")
            # Fall through to sequential processing
    
    # Sequential processing (original implementation)
    docs = []
    iterator = all_entries
    if show_progress:
        iterator = tqdm(all_entries, desc="Cleaning KWIC contexts")
    
    for entry in iterator:
        if isinstance(entry, dict):
            ctx = entry.get('context', entry.get('full_text', ''))
        else:
            ctx = str(entry)
        
        cleaned = clean_context(ctx, nlp=nlp, stopwords_set=stopwords_set,
                               keep_keyword=keep_keyword, min_len=min_len)
        if cleaned:
            docs.append(cleaned)
    
    return docs

In [ ]:
import random

random_keys = random.sample(list(bovine_kwic.keys()), 100)
filtered_kwic = {k: bovine_kwic[k] for k in random_keys}
len(filtered_kwic)

5

In [27]:
# Load spaCy model once
nlp = load_spacy()

# Optionally build stopwords (add domain-specific removals here)
stops = build_stopwords(extra=['court','section'])

# Use multiprocessing for faster processing
proc_kwic = kwic_dict_to_docs(bovine_kwic, nlp=nlp, stopwords_set=stops, 
                             keep_keyword='bank', parallel=True, 
                             show_progress=True,
                             chunk_size=500,
                             n_jobs=9)

Cleaning KWIC contexts (parallel): 100%|██████████| 98723/98723 [19:31<00:00, 84.30it/s] 


In [29]:
# Flatten bovine_kwic to get the original context strings
contexts = []
for entries in bovine_kwic.values():
    for item in entries:
        if isinstance(item, dict) and 'full_text' in item:
            contexts.append(item['full_text'])
        else:
            contexts.append(str(item))

# Create DataFrame
df_kwic = pd.DataFrame({
    'clean': contexts,
    'processed': proc_kwic
})
df_kwic.index.name = 'index'
df_kwic.head()

,clean,processed
index,,
0,They do n't wear uniforms with their names on ...,star army second lieutenant world war ii take ...
1,They do n't wear uniforms with their names on ...,ice building hotel three racetrack total asset...
2,They do n't wear uniforms with their names on ...,phillie million bank time say gile receive
3,They do n't wear uniforms with their names on ...,team remain hold canadian imperial bank commer...
4,She sensed what had happended the instant that...,plane see lie barely eet ground roil bank clou...


In [31]:
vectorizer = CountVectorizer(token_pattern="\\b[a-z][a-z]+\\b",
                             binary=True,
                             stop_words='english')

In [32]:
dtm_tf = vectorizer.fit_transform(df_kwic.processed)
print(dtm_tf.shape)

(98723, 57203)


### 2 Topics

- west bank in palestine (river bank), etc versus financial bank, money, loan, etc

In [41]:
%%time

lda_2 = LatentDirichletAllocation(n_components=2, random_state=42)
lda_2.fit(dtm_tf)

CPU times: user 1min 44s, sys: 734 ms, total: 1min 45s
Wall time: 1min 45s


,n_components,2
,doc_topic_prior,None
,topic_word_prior,None
,learning_method,'batch'
,learning_decay,0.7
,learning_offset,10.0
,max_iter,10
,batch_size,128
,evaluate_every,-1
,total_samples,1000000.0
,perp_tol,0.1


In [42]:
pyLDAvis.lda_model.prepare(lda_2, dtm_tf, vectorizer)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.125218  0.0       1        1  50.938799
1     -0.125218  0.0       2        1  49.061201, topic_info=          Term         Freq        Total Category  logprob  loglift
55324     west  7230.000000  7230.000000  Default  30.0000  30.0000
56122    world  6400.000000  6400.000000  Default  29.0000  29.0000
8640   central  5270.000000  5270.000000  Default  28.0000  28.0000
15537   ederal  2834.000000  2834.000000  Default  27.0000  27.0000
42181  reserve  2360.000000  2360.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
56629     year  2219.630173  4633.737770   Topic2  -5.6051  -0.0239
316    account  2720.827403  7408.370554   Topic2  -5.4015  -0.2896
1757   america  1977.186325  3671.955702   Topic2  -5.7207   0.0931
50841     time  1659.274839  3204.822279   Topic2  -5.8960   0.0538
28360     loan  1770.586311  4118.433146   Topic2  -5.8311  -0.1321

[175 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
316        1  0.632798  account
316        2  0.367287  account
482        1  0.934215      act
482        2  0.065735      act
1757       1  0.461607  america
...      ...       ...      ...
56122      2  0.000781    world
56627      1  0.001641     yeah
56627      2  0.999668     yeah
56629      1  0.520962     year
56629      2  0.479095     year

[267 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

### 4 Topics

In [33]:
%%time

lda_4 = LatentDirichletAllocation(n_components=4, random_state=42)
lda_4.fit(dtm_tf)

CPU times: user 1min 34s, sys: 512 ms, total: 1min 34s
Wall time: 1min 35s


,n_components,4
,doc_topic_prior,None
,topic_word_prior,None
,learning_method,'batch'
,learning_decay,0.7
,learning_offset,10.0
,max_iter,10
,batch_size,128
,evaluate_every,-1
,total_samples,1000000.0
,perp_tol,0.1


In [34]:
pyLDAvis.lda_model.prepare(lda_4, dtm_tf, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.188914 -0.063669       1        1  28.438414
2     -0.000849  0.029350       2        1  24.668216
1     -0.026593  0.130447       3        1  24.633829
0     -0.161473 -0.096127       4        1  22.259541, topic_info=            Term         Freq        Total Category  logprob  loglift
55324       west  7246.000000  7246.000000  Default  30.0000  30.0000
56122      world  6426.000000  6426.000000  Default  29.0000  29.0000
316      account  7349.000000  7349.000000  Default  28.0000  28.0000
32056      money  6374.000000  6374.000000  Default  27.0000  27.0000
1757     america  3688.000000  3688.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
24326      issue   633.739273  1222.574934   Topic4  -6.0682   0.8453
53575        use   838.847836  3299.296547   Topic4  -5.7878   0.1330
44345        say  1002.097011  8919.619253   Topic4  -5.6100  -0.6838
39421  president   720.165442  2328.574422   Topic4  -5.9404   0.3289
47857      state   693.437533  2380.559378   Topic4  -5.9782   0.2690

[335 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
316        1  0.005850  account
316        2  0.871575  account
316        3  0.081770  account
316        4  0.040953  account
459        2  0.009868  acquire
...      ...       ...      ...
56776      1  0.012765     york
56776      2  0.000672     york
56776      3  0.640270     york
56776      4  0.346673     york
56987      4  0.995988  zealand

[678 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 2, 1])

### 6 Topics

In [35]:
%%time

lda_6 = LatentDirichletAllocation(n_components=6, random_state=42)
lda_6.fit(dtm_tf)

CPU times: user 1min 25s, sys: 844 ms, total: 1min 26s
Wall time: 1min 27s


,n_components,6
,doc_topic_prior,None
,topic_word_prior,None
,learning_method,'batch'
,learning_decay,0.7
,learning_offset,10.0
,max_iter,10
,batch_size,128
,evaluate_every,-1
,total_samples,1000000.0
,perp_tol,0.1


In [36]:
pyLDAvis.lda_model.prepare(lda_6, dtm_tf, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.178679 -0.044379       1        1  22.830705
1     -0.055168 -0.110551       2        1  18.937228
2      0.012294 -0.120433       3        1  18.011420
0     -0.171747  0.033583       4        1  15.074228
5      0.150133  0.138800       5        1  13.185716
4     -0.114191  0.102981       6        1  11.960703, topic_info=          Term         Freq        Total Category  logprob  loglift
55324     west  7071.000000  7071.000000  Default  30.0000  30.0000
8640   central  5302.000000  5302.000000  Default  29.0000  29.0000
316    account  7407.000000  7407.000000  Default  28.0000  28.0000
56122    world  6401.000000  6401.000000  Default  27.0000  27.0000
32056    money  6426.000000  6426.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
56122    world   613.211846  6401.265159   Topic6  -5.4800  -0.2220
37727  percent   434.541779  1673.460496   Topic6  -5.8244   0.7752
28802      low   406.335702  1142.632179   Topic6  -5.8915   1.0896
50102      ter   442.782341  2837.623733   Topic6  -5.8056   0.2659
32554       mr   408.504259  1596.739438   Topic6  -5.8862   0.7603

[468 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
316        1  0.107454         account
316        2  0.001215         account
316        3  0.889059         account
316        4  0.002295         account
318        4  0.992292  accountability
...      ...       ...             ...
56683      5  0.014893             yes
56776      1  0.090339            york
56776      2  0.560237            york
56776      4  0.318883            york
56776      6  0.030338            york

[1112 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 3, 1, 6, 5])

### 8 Topics

In [37]:
%%time

lda_8 = LatentDirichletAllocation(n_components=8, random_state=42)
lda_8.fit(dtm_tf)

CPU times: user 1min 17s, sys: 438 ms, total: 1min 17s
Wall time: 1min 17s


,n_components,8
,doc_topic_prior,None
,topic_word_prior,None
,learning_method,'batch'
,learning_decay,0.7
,learning_offset,10.0
,max_iter,10
,batch_size,128
,evaluate_every,-1
,total_samples,1000000.0
,perp_tol,0.1


In [38]:
pyLDAvis.lda_model.prepare(lda_8, dtm_tf, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.213941 -0.038217       1        1  20.813224
1     -0.060089 -0.103415       2        1  14.949634
6      0.081225  0.007891       3        1  13.158509
2     -0.014750 -0.095852       4        1  12.275318
0     -0.198601  0.007992       5        1  10.182333
3      0.073539 -0.061329       6        1   9.774915
4     -0.147071  0.043508       7        1   9.673703
5      0.051806  0.239423       8        1   9.172364, topic_info=          Term         Freq        Total Category  logprob  loglift
55324     west  7378.000000  7378.000000  Default  30.0000  30.0000
56122    world  6424.000000  6424.000000  Default  29.0000  29.0000
8640   central  5316.000000  5316.000000  Default  28.0000  28.0000
316    account  7342.000000  7342.000000  Default  27.0000  27.0000
32056    money  6338.000000  6338.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
35810     orce   343.526630   920.745209   Topic8  -5.7940   1.4031
9527      city   381.245855  1450.573890   Topic8  -5.6898   1.0527
44345      say   509.732784  8913.222969   Topic8  -5.3994  -0.4724
50102      ter   394.680966  2841.715370   Topic8  -5.6552   0.4149
10948  control   351.874939  1299.156782   Topic8  -5.7700   1.0828

[617 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
310        1  0.000776    accord
310        2  0.223375    accord
310        3  0.093073    accord
310        4  0.134180    accord
310        5  0.303263    accord
...      ...       ...       ...
56683      1  0.485058       yes
56683      4  0.003013       yes
56683      6  0.510667       yes
56776      3  0.999105      york
57126      3  0.967826  zoellick

[1740 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 2, 7, 3, 1, 4, 5, 6])

### 12 Topics

In [39]:
%%time

lda_12 = LatentDirichletAllocation(n_components=12, random_state=42)
lda_12.fit(dtm_tf)

CPU times: user 1min 15s, sys: 778 ms, total: 1min 16s
Wall time: 1min 17s


,n_components,12
,doc_topic_prior,None
,topic_word_prior,None
,learning_method,'batch'
,learning_decay,0.7
,learning_offset,10.0
,max_iter,10
,batch_size,128
,evaluate_every,-1
,total_samples,1000000.0
,perp_tol,0.1


In [40]:
pyLDAvis.lda_model.prepare(lda_12, dtm_tf, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.205287  0.007177       1        1  15.864577
1      0.072107  0.106580       2        1  11.536858
6     -0.052598 -0.027056       3        1   9.265542
8      0.170634 -0.069313       4        1   8.943173
10    -0.161703  0.033884       5        1   8.198542
5      0.000103 -0.233222       6        1   7.849011
9      0.018906  0.093989       7        1   7.817079
4      0.158428 -0.055792       8        1   7.577200
2      0.010478  0.119026       9        1   7.505494
0      0.165804  0.074243      10        1   6.381870
3     -0.109767  0.002864      11        1   5.623269
11    -0.067104 -0.052381      12        1   3.437386, topic_info=           Term         Freq         Total Category  logprob  loglift
55324      west  7330.000000   7330.000000  Default  30.0000  30.0000
56122     world  6333.000000   6333.000000  Default  29.0000  29.0000
8640    central  5310.000000   5310.000000  Default  28.0000  28.0000
316     account  7394.000000   7394.000000  Default  27.0000  27.0000
32056     money  6378.000000   6378.000000  Default  26.0000  26.0000
...         ...          ...           ...      ...      ...      ...
22957  inancial   141.646096   2643.580315  Topic12  -5.6985   0.4439
44345       say   170.868698   8930.389060  Topic12  -5.5109  -0.5859
43200       rom   171.060548  11243.226619  Topic12  -5.5098  -0.8150
5499        big   128.298880   1795.364017  Topic12  -5.7974   0.7319
42848      risk   115.243603    737.969083  Topic12  -5.9048   1.5136

[917 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
139       12  0.967741       abn
279        2  0.033805    access
279        4  0.140289    access
279        5  0.155501    access
279        6  0.028734    access
...      ...       ...       ...
56766      3  0.961282      yong
56776      3  0.999562      york
56777      3  0.976290  yorkbase
57058      8  0.962514      zhou
57126      3  0.966999  zoellick

[3080 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 2, 7, 9, 11, 6, 10, 5, 3, 1, 4, 12])